In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, multivariate_normal
from scipy.optimize import minimize

# =============================
# PARAMETERS
# =============================

R = 0.4          # recovery rate
N = 120          # number of names
T = 5.0          # maturity (years)
r = 0.0014       # risk-free rate
M = 30000        # Monte Carlo samples (increase for stability)

payment_times = np.arange(1, int(T) + 1)

# Tranches (attachment, detachment)
tranches = [
    (0.00, 0.03),
    (0.03, 0.07),
    (0.07, 0.15),
    (0.15, 1.00),
]

# =============================
# DATA
# =============================

data = {
    "Date": [
        "2014-06-01", "2014-07-03", "2014-08-15", "2014-09-23",
        "2014-10-11", "2014-11-17", "2014-12-01",
        "2015-01-07", "2015-02-10", "2015-03-15"
    ],
    "Tranche_0_3": [4.250, 3.750, 4.094, 3.750, 5.775, 4.188, 3.183, 7.065, 7.559, 6.874],
    "Tranche_3_7": [2.000, 1.375, 1.719, 1.375, 1.810, 0.985, 0.747, 0.875, 0.563, 0.073],
    "Tranche_7_15": [0.036, 0.048, 0.050, 0.056, 0.050, 0.057, 0.060, 0.055, 0.055, 0.064],
    "Tranche_15_100": [0.014, 0.015, 0.014, 0.012, 0.012, 0.015, 0.016, 0.013, 0.014, 0.015],
    "CDS_Index": [39, 37, 38, 37, 41, 35, 32, 39, 37, 34]
}

df = pd.DataFrame(data)

# =============================
# UTILITIES
# =============================

def hazard_from_cds(spread_bps):
    """
    lambda ≈ spread / (1 - R)
    """
    return (spread_bps * 1e-4) / (1.0 - R)


def discount(t):
    return np.exp(-r * t)


def gaussian_copula_samples(rho, n_samples, dim):
    """
    Exchangeable Gaussian copula
    """
    corr = rho * np.ones((dim, dim)) + (1 - rho) * np.eye(dim)
    z = multivariate_normal.rvs(mean=np.zeros(dim), cov=corr, size=n_samples)
    return norm.cdf(z)


def tranche_loss(portfolio_loss, a, d):
    return np.minimum(np.maximum(portfolio_loss - a, 0.0), d - a)


# =============================
# PRICING
# =============================

def tranche_spreads_model(rho, lambda_i):
    """
    Compute tranche spreads for a given rho and hazard rate lambda
    """
    U = gaussian_copula_samples(rho, M, N)
    tau = -np.log(U) / lambda_i

    spreads = []

    for a, d in tranches:
        L_prev = np.zeros(M)
        num = 0.0
        den = 0.0

        for t in payment_times:
            defaults = (tau <= t).sum(axis=1)
            L = (1 - R) * defaults / N
            Lt = tranche_loss(L, a, d)

            # Default leg increment
            num += discount(t) * np.mean(Lt - L_prev)

            # Outstanding notional
            P_prev = d - a - L_prev
            P_t = d - a - Lt

            den += discount(t) * np.mean(0.5 * (P_prev + P_t))

            L_prev = Lt

        spreads.append(num / den)

    return np.array(spreads)


# =============================
# CALIBRATION
# =============================

market_tranches = df[[
    "Tranche_0_3",
    "Tranche_3_7",
    "Tranche_7_15",
    "Tranche_15_100"
]].values * 1e-2  # convert to decimal


def objective(params):
    rho = params[0]
    errors = []

    for i in range(len(df)):
        lambda_i = hazard_from_cds(df.loc[i, "CDS_Index"])
        model_spreads = tranche_spreads_model(rho, lambda_i)
        errors.append(model_spreads - market_tranches[i])

    errors = np.concatenate(errors)
    return np.sqrt(np.mean(errors ** 2))


res = minimize(
    objective,
    x0=[0.2],
    bounds=[(0.01, 0.99)],
    method="L-BFGS-B"
)

print("Calibrated Gaussian copula correlation rho =", res.x[0])
print("RMSE =", res.fun)


Calibrated Gaussian copula correlation rho = 0.19999953818872646
RMSE = 0.03395701566244567
